**Course 3 -> Algorithms for DNA Sequencing**

Course link: <https://www.coursera.org/learn/dna-sequencing?specialization=genomic-data-science>

---

*Ideas behind* algorithms.

> Came back from Course 4 and realized that the lecturer for Course 3 is the creator of Bowtie :O

- **2 foci**: the read **alignment** problem (reference genome present) and the genome **assembly** problem (reference genome absent, *de novo*; HGP's shotgun.) 

- **2 formats**: 

  - **FASTA**: stores the *reference genome* that the sequence fragments/*reads* will be mapped to via alignment. Starts with a header line ('>' and description). No blank lines allowed.
  
    - Multi-FASTA: contains multiple sequences (eg. all chromosomes), each followed immediately by the header line of the next sequence. No blank sequences or any line that starts with a space allowed.


  - **FASTQ**: format for NGS. Stores the sequence reads before mapping.
 
    - Each record contains four lines: 1. header (preceded by '@'); 2. the sequence; 3. just a placeholder line, can be omitted; 4. the base quality scores reflecting reliability. Each base quality (the letter) is an ASCII encoded representation of that value Q. Q is calculated (taking the negative log (which is kind of like how pH is calculated) and then times 10) from p, *the probability that the base call is incorrect*. So when Q is higher, we can be more confident that the base is correct.

      - We can look at a ASCII table, which shows the mapping between characters and integers. This encoding is done through a method called Phred 33. 

      - We stack together many such records for the reads in a FASTQ file. 
  
```Python
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline() # skip name line
            seq = fh.readline().rstrip() # read base sequence
            fh.readline() # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities
```
   

Some bases in the reads differ from the reference genome due to several reasons like sequencing errors, polymorphism, etc. We have to take into account these inevitable differences in both of the two problems (alignment and assembly). 



## Focus 1: Read *Alignment*

### Exact matching

![Exact matching algorithms](https://freeimghost.net/images/2025/04/30/Screenshot-2025-05-01-at-08.41.07.png)

#### Naive exact matching 

A preliminary idea for dealing with such task. Simply consists of two nested loops, a loop that iterates over possible alignments and then a loop that iterates over character comparisons, both searched from *left to right*. We're just trying every possible offset and checking whether P ('pattern', eg. short read) matches T ('text', eg. full genome) at that offset; if characters on the corresponding position mismatch, the inner loop `break`s.

#### Boyer-Moore

For a given alignment (also aligned in left-to-right order), we will do the **character comparisons** but **starting from the right-hand side** and comparing each character to see whether they mismatch. 

- Upon mismatch, we can skip alignments until either the mismatch becomes a match, or the pattern P moves all the way past the mismatched text character. This is called the *bad character rule*. The other rule is called the *good suffix rule*, which states that for the characters we did match (P's *good suffix*), we want to shift P until there are no mismatches between P and common substring, or until P moves all the way past the substring on T.

-  Whenever we have a mismatch, we're going to try both rules. **Each rule will tell us some number of alignments that we can skip**. And we take the maximum of the two to skip.

-  In reality, Boyer-Moore implementations build some **lookup tables beforehand** (*preprocess* P), so that every time we want to apply either rule, the number of skips can simply be looked up from the table.
  
#### Offline algorithms: indexing

Boyer-Moore is an algorithm that preprocesses the pattern P to build the lookup tables. However, if we're in a situation where we have to solve many matching problems in which the text T is the same but the pattern P can vary from problem to problem, then it might be worthwhile to preprocess the text T. 

An algorithm that uses a **preprocessed version of T** is called an **offline algorithm**, whereas an algorithm that does not preprocess T is called online, eg. Boyer-Moore. Notice that this definition is the same regardless of whether we preprocess P or not.

**Indexing**: two ways, either by **sorting** (eg. in alphabetical order) or by **grouping** (eg. by category).

We index our text T in pretty much the same way we would index a book, except that we treat consecutive **substrings of T with the length of k** as words, given the term ***k-mer***. We take every k-mer and insert it into the index while *recording its offset* at T, then sort it by alphabetical order so that for every k-mer we are able to map it back to all the offsets where it occurred. 

- For a P, we query this index by taking the k-mer of P and matching it with the index to get the offset within T.

- After finding the index hits, the place where the k-mer occurs within the text, that's only the first step. The additional work we have to do to determine if we have a **full match of P to T is called verification**. We basically compare the rest of the characters of P other than k-mer with T. 

- This way to store the index is called a **multimap**, since a k-mer may be associated with many different offsets in the genome. 

  - A data sturcture based on *ordering*: an **ordered list** of k-mer-offset pairs ordered alphabetically by k-mer. We can use binary search (supported by Python's `bisect` module) to query this (since it's ordered!!!).


  - Another data structure to implement a multimap but based on *grouping*: **hash tables**. They are more or less the standard data structure used to represent sets and maps in practice (**Python's dictionary is just an implementation of a hash table**). 

    - An empty hash table consists of an array of buckets, and the buckets start out as empty boxes that can be seen as null references or null pointers.
   
    - We use the hash function to assign an entry to a bucket. A given bucket is simply a **linked list**; to add our key-value pair, we're simply appending an entry onto that linked list. The entry consists of a key (the k-mer) and a value (the offset from which we got that k-mer). And then there's a null reference; if we add some more entries later, we are adding them onto the end of the linked list by expanding on this null reference.

      -  We don't know what hash function is being used or exactly how many elements are in the hash table. All of that is taken care of by Python. But when we use a dictionary in Python, we should know that it's basically doing the same thing like the hash table described here.

    - Once we have more possible k-mers than we have buckets, the *pigeonhole principle* expects some distinct k-mers to end up in the same bucket together, called a *collision*. When there are many collisions, querying the data structure can become somewhat slower.

**Two variations of k-mer indexing** (for optimization):

1. Instead of taking k-mers at consecutive offsets, we take every **n-th** k-mer in T. This makes the index smaller, which increases efficiecy.

  - To avoid missing hits and compensate for this smaller index of T, when we *query* using P, we have to take **n** k-mers on P whose **offsets are not congruent to modulo n**.

2. Instead of using the substrings of T to build the index, we use **subsequences** of T, and then use subsequences concatenated by characters at the corresponding positions from P to query. 

  - Subsequences still consist of characters from a string and in the same order, but **not necessarily consecutively**, which distinguishes them from substrings. Eg. a subsequence con consists of characters from a string at offset 1, 5, and 7 and then concatenated together. 

  - The advantage of this variation is that the index filter tends to be *more specific*. Therefore, when we get an index hit, it's going to lead to a more successful verification than if we had taken those characters to be consecutive (i.e. built over sub*strings*). 

**Suffix indices for genome indexing**:

- Suffix array: stores the offset (so that it's only a list (of the same length as the genome) that stores integers) of all the *suffixes with different lengths* on the genome, sorted by alphabetical order of their individual prefixes. The full string of the genome is also required to accompany the interger list. 

- Suffix tree: organizes all of the suffixes of T using the principle of *grouping* rather than the principle of *ordering*, which is the case for suffix arrays as they require to be sorted alphabetically.

- **FM index**: rlly compact, which is why it is widely used and is the basis for some read alignment tools like Bowtie and BWA covered in Course 4. Based on another idea called the Burrows-Wheeler transform (BWT), which was originally developed for compression.

### Approximate matching

As mentioned earlier, differences between read and reference are practically inevitable due to sequencing error and natural variation between organisms. So exact matching (covered above) isn't so practical, which is why we now turn to... \*drum roll*

**Approximate matching**: finding approximate occurrences of a pattern P (sequenced reads) in a text T (the reference genome). 

- The pigeonhole principle.

- Dynamic programming. 

#### Measuring distances between strings

- **Hamming distance**: for strings with the same length, the minimum number of substitutions needed.

- **Edit/Levenshtein distance**: d/n require identical lengths; finds the minimum number of edits (substitutions **and indels**. The latter allows us to get from X to Y with potentially *fewer* changes.)

#### A naive idea

Recall the naive *exact* matching. Now we'd like to adapt this simple algorithm to allow mismatches: we want to look for occurrences of the pattern P within the text T that are **within some specified Hamming distance** (exact matches are within a hamming distance of 0). We can achieve this by preventing the inner loop from immediately `break`ing as soon as we encounter a single mismatch, but rather *keeping track of the encountered mismatches* and only `break` out of the inner loop when it exceeds the maximum number of mismatches allowed.

#### The pigeonhole principle

If P occurs in T with up to *k edits*, then if we divide P into *(k + 1)* **partitions**, *at least one of these partitions of P must appear with zero edits, i.e. an exact match*. And then we can use some exact matching algorithm, say Boyer-Moore or an index-assisted algorithm, to look for the *exact match* in those (k + 1) different partitions. 

- The **verification** step is also required: when we find a *partition that matches exactly* within T, we have to verify whether *the entire string P* occurs within *the vicinity of that partition hit with up to the specified maximum number of edits allowed*.


In [ ]:
# The BoyerMoore class and the boyer_moore function are pre-defined in the practical session.
# I only pasted the function for approximate matching here to help understand.
def approximate_match(p, t, max_edits):
    partition_length = int(round(len(p) / (max_edits+1)))
    all_matches = set()
    for i in range(max_edits+1):
        start = i * partition_length
        end = min((i+1) * partition_length, len(p))
        p_bm = BoyerMoore(p[start:end], alphabet='ACGT')
        matches = boyer_moore(p[start:end], p_bm, t)
        # Verification step: extend matched partition to see if whole p matches within max_edits.
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > max_edits:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > max_edits:
                        break
            if mismatches <= max_edits:
                all_matches.add(m - start)
    return list(all_matches)

#### Dynamic programming and the edit distance algorithm

A **dynamic programming** algorithm: breaking a complex problem (i.e., finding the edit distance) into simpler subproblems, and using the **stored** solutions *to these subproblems* to solve the original problem.

The basic idea here is to create a **matrix** where we sequentially fill in the edit distances for the substrings we have *already* computed (the *stored solutions*), so we don't have to recompute them multiple times, as in recursion.

- The dynamic programming matrix: the rows are labelled with characters from the read (P), and the columns are labelled with characters from the genome (T). Then we fill in the values in order (top-down). 

- 'Traceback': retracing our steps to figure out how we got to the final edit distance that was placed in that cell in the bottom row of the matrix. We use this to **identify the exact location of the match on T and the shape (eg. gaps) of the alignment**. 

![Traceback](https://freeimghost.net/images/2025/04/30/Screenshot-2025-05-01-at-09.11.07.png)

- Limitation: the matrix can be massive due to the length of T (i.e. the entire reference genome). So in practice, other algorithms are often used in combination, eg. indexing, which is fast and good at narrowing down the places to look for matches (tho it doesn't handle mismatches and gaps naturally, so we need dynamic programming (like the two algorithms below!) for the **verification step** in a alignment task.

- **Global alignments and local alignments**.


In [5]:
import time

# If we don't use dynamic programming, the edit distance algorithm via *recursion* is extremely slow,
# since it repeatedly/recursively calculates the same subproblems and does not store the solutions every time.
def editDistRecursive(x, y):
    if len(x) == 0:
        return len(y)
    elif len(y) == 0:
        return len(x)
    else:
        distHor = editDistRecursive(x[:-1], y) + 1
        distVer = editDistRecursive(x, y[:-1]) + 1
        if x[-1] == y[-1]:
            distDiag = editDistRecursive(x[:-1], y[:-1])
        else:
            distDiag = editDistRecursive(x[:-1], y[:-1]) + 1
        return min(distHor, distVer, distDiag)
    

# Using the dynamic programming matrix:
def editDistance(x, y):
    # Create matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

start = time.time()
x = 'shake spea'
y = 'Shakespear'
editDistRecursive(x, y)
end = time.time()
print(f"Recursive edit distance: {end - start}s")

start2 = time.time()
x = 'shake spea'
y = 'Shakespear'
editDistance(x, y)
end2 = time.time()
print(f"Dynamic programming edit distance: {end2 - start2}s")

Recursive edit distance: 2.3921008110046387s
Dynamic programming edit distance: 0.0001270771026611328s


#### Global alignments and local alignments

**Global alignment**: very much analogous to the edit distance algorithm, but it involves the **penalty matrix, which accounts for frequency differences of mutation types.**

When computing the distance for each change in the string, here we assign different penalties (weights) to different types of mutations (since they cause our strings/DNA sequences to change). Eg:

- Indels/gaps are less frequent than substitutions. So we would want to *penalize indels more than substitutions*.

- In substitutions, transitions are more frequent than transversions (the ratio of these is ~2.1 for humans). 

The penalty matrix is used in the algorithm when we're combining the effects of the three previous contributions (diagonal, vertical, and horizontal). In the original case for edit distance, we simply add 1 for a vertical or a horizontal move on the *dynamic programming matrix* and 0/1 (whether the characters from two strings are equal) for a diagonal move, but in this case we **add a value that we look up in the penalty matrix**. 

- Apart from this, the algorithm is pretty much identical to the algorithm for edit distance, both in terms of how we fill the dynamic programming matrix and also in terms of how we find the traceback.

---

**Local alignment**, on the other hand, is a somewhat different kind of problem. 

Here we're not trying to find the distance between two strings, and we're not trying to find occurrences of one string inside another. The problem we're trying to solve is, given two strings X and Y, we'd like to **identify the substring of X and the substring of Y that are most similar to each other!!!**

Instead of using a penalty matrix, we're going to use something similar called the **scoring matrix**. In this matrix, we **give a positive bonus for a match and negative penalties to different kinds of edits**. Eg. matches score +2, whereas substitutions have a penalty of -4 and gaps have a penalty of -6. 

Again we'll fill in a dynamic programming matrix, by taking the *maximum* of the corresponding values looked up from the scoring matrix and *0*. We'll then notice that many of the elements in the DP matrix are 0. 

- **The goal of local alignment is to find parts of X and Y that match closely enough that they sort of pop out from the background of 0s (dissimilarities) in the DP matrix**. So we're going to look for the element in the matrix that has the maximal value. 

  - And then if we want to know which substrings are involved in that alignment, and exactly what the shape of that alignment is, we can just use our **traceback** procedure as described above in the section on DP. The only thing we'll modify our typical traceback procedure is that we stop as soon as we reach an element whose value is 0.


### Concluding the read alignment problem

In practice, the tools used for the ***alignment*** problem are usually based on the ideas/algorithms we've covered, especially **indexing (for filtering) and dynamic programming (for the verification step**, to see whether the pattern as a whole has an approximate match to the vicinity of that index hit). 



## Focus 2: Genome *Assembly*

The *overlaps of reads* are the *glue* we use to assemble the genome.

**'Coverage'**: the counts of reads that cover a genomic position.

- Overall coverage: the coverage averaged over all the positions of the genome. So we can calculate this by taking the *total length of all the reads* and dividing it by the total length of the genome.

- The more coverage we have, the more and longer the overlaps we have between reads (and thus more *glue* for assembly).

**Overlap graphs**: 

- Nodes represent overlapped substrings (k-mers).

- Directed edges point from the **suffix of the former substring to the prefix of the latter**. 

- Each edge is labeled with the length of the corresponding suffix-prefix match. Since again we have to account for the inevitable differences among reads, we set a threshold for the minimum length of overlap (to allow mismatches and gaps).

- On a simplified graph (since the graphs we get in practice would usually be very messy), we can walk along every node and then infer the sequence of the original genome.

In [ ]:
def overlap(a, b, min_length):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long. If no such overlap exists,
        return 0. """
    start = 0  # Start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # Look for b's prefix in a
        if start == -1:  # No more occurrences to right
            return 0
        # If occurrence found,
        # Check for suffix/prefix match to return length of overlap
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # Move just past previous match


# A *naive* way to find all overlaps in a set of reads and construct an overlap graph is to 
# enumerate all possible pairs of reads by permutation, and check for overlaps.
from itertools import permutations
def naive_overlap_map(reads, min_length):
    """ Return a dictionary of all overlaps in a set of reads.
        The keys are tuples of the two reads that overlap and the values are the lengths
        of the overlaps. """
    olaps = {}
    for a, b in permutations(reads, 2):
        olen = overlap(a, b, min_length)
        if olen > 0:
            olaps[(a, b)] = olen
    return olaps

reads = ['ACGGATC', 'GATCAAGT', 'TTCACGGA']
print(naive_overlap_map(reads, 3))

{('ACGGATC', 'GATCAAGT'): 4, ('TTCACGGA', 'ACGGATC'): 5}


### Shortest common superstring (SCS) 

Given a set of input strings, find the shortest string that contains all of the input strings as substrings.

- Can be found by brute force, like listing out all of the **permutations** of these input strings and glue the overlapped adjacent pairs. 

  - 'NP-complete': no efficient algorithm for large inputs.

#### Greedy SCS 

Stores the best overlap length. However, it can sometimes report a superstring that's not necessarily the shortest.

- Consider an original string that contains copies of the same substring. Conceivably, if we try to find it using SCS by merging the k-mers of this string that cover the copy, the returned superstring might even be shorter than the original string since the repetitive copies in the k-mers are merged as well and disappear. 

  - This is very undesirable in assembly tasks since the genome is so extremely rich in repetitive sequences. It will just assemble fewer copies of the repeats.

### De Bruijn graph and Eulerian walks

An alternative approach to SCS, which is problematic when the genome is repetitive. 

To construct a De Bruijn graph, we first have to assume something about the sequencing reads: each of them are of length k (i.e. **our reads consist of each of the k-mers from the genome**), and **each k-mer is sequenced once**. So for every offset in the genome, we're going to extract that k-mer; our sequencing output is just going to be a list of all the k-mers in the genome.

![De Bruijn graph](https://freeimghost.net/images/2025/04/30/Screenshot-2025-05-01-at-10.42.51.png)

Under these (unrealistic) assumptions, in a De Bruijn graph constructed, we walk across **each edge exactly once**, giving a reconstruction of the genome (the long string). This is called an Eulerian walk. It would not overcollapse (eat) the repeats, which is an improvement over the SCS formulation of the assembly problem. 

However, it is still flawed when dealing with repetitive sequences: there might be multiple ways to do the Eulerian walks, and only one of those walks corresponds to the original genomic sequence. Others involve incorrect reshufflings of repetitive sequences put in the wrong order.

Assemblers in practice: instead of reporting the final genome, report a set of **contigs**. 

Dealing with repetitive sequences: making the sequencing reads longer -> **anchor repetitive sequences to their surrounding nonrepetitive context**. The future is *long*. 